# Mo Ru Surrogate Model using Neural Networks

In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from matplotlib import pyplot as plt

### Read and Dump JSON

In [ ]:
import json

filename = 'data/MoRu_sparse.json'
stream = open(filename)
data = json.load(stream)
stream.close()

### Generate Dataset from JSON Dump

The dataset $\mathcal{D} \subset \mathbb{R}_{\ge 0}^{n_\text{samples} \times (n_\text{elements} + n_\text{phases} + 2)}$ is defined as:
$$\mathcal{D} = \{\mathbf{x}_i \, | \, i \in \{1, \dots, n_\text{samples}\}\}$$
where, $\mathbf{x}_i$ is a single sample such that:
$$\mathbf{x}_i = \left\{{T, P, \{c_j \, | \, j \in \mathcal{1, \dots, n_\text{elements}}\}, \{n_\phi \, | \, \phi \in \mathcal{1, \dots, n_\text{phases}}\}}\right\}$$
where, $c_j$ denotes number of moles of element $j$ and $n_\phi$ denotes the number of moles of phase $\phi$.

In [ ]:
n_data = len(data)
n_elements = len(data['2']['elements'])
n_phases = len(data['2']['solution phases']) + len(data['2']['pure condensed phases'])
dataset = np.zeros((n_data, n_elements + n_phases + 2))

phase_names = list(data['2']['solution phases'].keys())
phase_names += list(data['2']['pure condensed phases'].keys())
element_names = list(data['2']['elements'].keys())

In [ ]:
keys = list(data.keys())
for i in keys:
    dataset[keys.index(i), 0] = data[i]["temperature"]
    dataset[keys.index(i), 1] = data[i]["pressure"]

    for j in range(n_elements):
        if not (data[i]['elements'].get(element_names[j]) is None):
            dataset[keys.index(i), 2 + j] = data[i]["elements"][element_names[j]]["moles"]
        else:
            dataset[keys.index(i), 2 + j] = 0.0

    for j in range(n_phases):
        if not (data[i]['solution phases'].get(phase_names[j]) is None):
            dataset[keys.index(i), 2 + n_elements + j] = data[i]["solution phases"][phase_names[j]]["moles"]
        elif not (data[i]['pure condensed phases'].get(phase_names[j]) is None):
            dataset[keys.index(i), 2 + n_elements + j] = data[i]["pure condensed phases"][phase_names[j]]["moles"]
        else:
            dataset[keys.index(i), 2 + n_elements + j]

### Generate Training Data

- Training samples $n_\text{Train} = 0.6 * n_\text{Total}$
- Input $\mathbf{X} = {T, P, \mathbf{c}}$
- Output $\mathbf{y} = {\mathbf{n}_\phi}$

In [ ]:
n_training = int(n_data * 0.6)

choice = np.random.choice(range(dataset.shape[0]), size=(n_training,), replace=False)
train_indices = np.zeros(dataset.shape[0], dtype=bool)
train_indices[choice] = True
test_indices = ~train_indices


train = dataset[train_indices]
test = dataset[test_indices]

X_train = train[:,0:(2 + n_elements)]

# Change the outputs to 0.0 / 1.0 based on the number of moles of phase
y_train = (train[:,(2 + n_elements):] > 0.0).astype(float)

# Use the original moles 
# y_train = train[:,(2 + n_elements):]

In [ ]:
# Convert to Torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_train_unscaled = X_train.detach().clone()
y_train = torch.tensor(y_train, dtype=torch.float32)
y_train_unscaled = y_train.detach().clone()

In [ ]:
# Calculate mean and standard deviation of inputs
X_mean = X_train.mean(0,keepdim=True)
X_std = X_train.std(0,keepdim=True)

# Standard deviation of pressure is equal to 0.0. Change this to 1.0 to avoid division by zero when scaling
X_std[X_std == 0] = 1.0

# Scale inputs to mean 0 and standard deviation 1 for better convergence
X_train = (X_train - X_mean) / X_std

### Test Data

In [ ]:
# Scale test input values to mean 0 and standard deviation 1 based on values from before
X_test_unscaled = torch.tensor(test[:,0:(2 + n_elements)], dtype=torch.float32)
X_test = (X_test_unscaled - X_mean) / X_std

# 0 / 1 Output case
y_test = (test[:,(2 + n_elements):] > 0.0).astype(float)

# Original moles of phase
# y_test = test[:,(2 + n_elements):]
y_test = torch.tensor(y_test, dtype=torch.float32)

# Combined test and training data
X_all_unscaled = torch.cat([X_train_unscaled, X_test_unscaled])
X_all = torch.cat([X_train, X_test])
y_all = torch.cat([y_train, y_test])

### Neural Network Model

In [ ]:
torch.device('mps')

In [ ]:
input_size = X_test.shape[1]
hidden_size = [10, 10]
output_size = y_test.shape[1]

model = nn.Sequential(
    nn.Linear(input_size, hidden_size[0]),
    nn.ReLU(),
    nn.Linear(hidden_size[0], hidden_size[1]),
    nn.ReLU(),
    nn.Linear(hidden_size[1], output_size),
    nn.Sigmoid())

In [ ]:
loss_fn = nn.MSELoss()  # Mean Square Error
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training

In [ ]:
n_epochs = 10000
batch_size = 100
 
for epoch in range(n_epochs):
    for i in range(0, len(X_train), batch_size):
        Xbatch = X_train[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y_train[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

###  Evaluation

In [ ]:
# Compute accuracy for NN data set
with torch.no_grad():
    y_pred = model(X_all)

diff = (y_pred - y_all)
accuracy =(diff**2).mean()**.5
delta_min = diff.min()
delta_max = diff.max()
print(f"NN Absolute RMS Accuracy {accuracy},  Min {delta_min}, Max {delta_max}")

In [ ]:
# Creating figure
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
 
# Evaluate the model
with torch.no_grad():
    y_pred = model(X_train)

# Creating plot
ax.scatter3D(X_train_unscaled[:,0], X_train_unscaled[:,2], y_train_unscaled[:,5], marker = 's', color = "green", alpha = 0.65, label = "Input")
ax.scatter3D(X_train_unscaled[:,0], X_train_unscaled[:,2], y_pred[:,5], color = "red", alpha = 0.35, label = "Output")
plt.title("Moles of HCPN (Training)")
plt.xlabel("T")
plt.ylabel("Moles Mo")
plt.legend()
plt.show()

In [ ]:
# Creating figure
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
 
# Evaluate the model
with torch.no_grad():
    y_pred = model(X_test)

# Creating plot
X_test_unscaled = X_test * X_std + X_mean
ax.scatter3D(X_test_unscaled[:,0], X_test_unscaled[:,2], y_test[:,5], marker = 's', color = "green", alpha = 0.65, label = "Input")
ax.scatter3D(X_test_unscaled[:,0], X_test_unscaled[:,2], y_pred[:,5], color = "red", alpha = 0.35, label = "Output")
plt.title("Moles of HCPN (Test)")
plt.xlabel("T")
plt.ylabel("Moles Mo")
plt.legend()
plt.show()


In [ ]:
# Creating figure
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
ax.view_init(elev=10, azim=-30, roll=0)
 
# Evaluate the model
with torch.no_grad():
    y_pred1 = model(X_train)
    y_pred2 = model(X_test)

# NN error
y_pred = torch.cat([y_pred1, y_pred2])
y_err = (y_all - y_pred)

sc = ax.scatter3D(X_all_unscaled[:,0], X_all_unscaled[:,2], y_err[:,5], c=y_err[:,5], cmap='hsv')
fig.colorbar(sc, ax=ax)

plt.title("Moles HCPN (absolute error)")
plt.xlabel("T")
plt.ylabel("Moles Mo")
plt.show()